### Process UNSEEN ###

But in notebook form.

In [2]:
! conda install iris

Channels:
 - conda-forge
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.9.2

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /apps/jasmin/jaspy/miniforge_envs/jaspy3.11/mf3-23.11.0-0

  added / updated specs:
    - iris


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cartopy-0.24.0             |  py310h5eaa309_0         1.3 MB  conda-forge
    cf-units-3.2.0             |  py310hf462985_6         293 KB  conda-forge
    cftime-1.6.4               |  py310hf462985_1         243 KB  conda-forge
    cloudpickle-3.1.0          |     pyhd8ed1ab_1          25 KB  conda-forge
    contourpy-1.3.0            |  py310h3788b33_2         255 KB  conda-forge
    dask-core-2024.10.0        |     pyhd8ed1ab_0         879 K

In [1]:
%matplotlib inline
%reload_ext autoreload

# Local imports
import os
import sys
import time
import argparse

# Third-party imports
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import shapely.geometry
import cartopy.io.shapereader as shpreader
import iris

# Specific imports
from tqdm import tqdm
from datetime import datetime, timedelta

# Load my specific functions
sys.path.append("/home/users/benhutch/unseen_functions")
import functions as funcs
import bias_adjust as ba

ModuleNotFoundError: No module named 'iris'

In [ ]:
# Hardcoded variables
model = "HadGEM3-GC31-MM"
experiment = "dcppA-hindcast"
freq = "Amon" # go back to using monthly data

# Set up the arguments
variable = "tas"
country = "United_Kingdom"
season = "ONDJFM"
first_year = 1960
last_year = 2018
model_fcst_year = 1
lead_year = "1-10"
detrend = True # True for temperature, false for wind speeds
bias_correct = "None" # No bias correction for tas months
percentile = 10

# Save directory
save_dir = "/gws/nopw/j04/canari/users/benhutch/plots/unseen"

# list of valid bias corrections
valid_bias_corrections = [
    "None",
    "linear_scaling",
    "variance_scaling",
    "quantile_mapping",
    "quantile_delta_mapping",
    "scaled_distribution_mapping",
]

# Set up the output directory for the dfs
output_dir_dfs = "/gws/nopw/j04/canari/users/benhutch/unseen/saved_dfs"

: 

In [ ]:
# if the bias correction is not in the valid bias corrections
if bias_correct not in valid_bias_corrections:
    raise ValueError(f"Bias correction {bias_correct} not recognised")

# set up the obs variable depending on the variable
if variable == "tas":
    obs_var = "t2m"
elif variable == "sfcWind":
    obs_var = "si10"
else:
    raise ValueError("Variable not recognised")

# Set up the months depending on the season
if season == "DJF":
    months = [12, 1, 2]
elif season == "NDJ":
    months = [11, 12, 1]
elif season == "OND":
    months = [10, 11, 12]
elif season == "JFM":
    months = [1, 2, 3]
elif season == "MAM":
    months = [3, 4, 5]
elif season == "JJA":
    months = [6, 7, 8]
elif season == "SON":
    months = [9, 10, 11]
elif season == "ONDJFM":
    months = [10, 11, 12, 1, 2, 3]
elif season == "NDJFM":
    months = [11, 12, 1, 2, 3]
else:
    raise ValueError("Season not recognised")

# Depending on the model forecast year
# set the leads to extract from the model
if model_fcst_year == 0 and season == "NDJFM":
    lead_months = [1, 2, 3, 4, 5]
elif model_fcst_year == 1 and season == "ONDJFM":
    lead_months = [12, 13, 14, 15, 16, 17]
elif model_fcst_year == 1 and season in ["OND", "NDJ", "DJF", "JFM"]:
    lead_months = [12, 13, 14, 15, 16, 17] # include all then subset later
else:
    raise ValueError("Model forecast year and season not recognised")

: 

In [ ]:
# Set up the name for the obs df
obs_df_name = f"ERA5_obs_{variable}_{country}_{season}_{first_year}_{last_year}.csv"

# Set up the name for the model df
model_df_name = f"{model}_{variable}_{country}_{season}_{first_year}_{last_year}_{experiment}_{freq}.csv"

# form the full paths for the dfs
obs_df_path = os.path.join(output_dir_dfs, obs_df_name)
model_df_path = os.path.join(output_dir_dfs, model_df_name)

: 

In [ ]:
# if the obs df exists and the model df exists
if os.path.exists(obs_df_path) and os.path.exists(model_df_path):
    print("Loading the observed and model dfs")

    # load the dfs
    obs_df = pd.read_csv(obs_df_path)
    model_df = pd.read_csv(model_df_path)

    # print("Loaded the dfs")
    # print("----------------")
    # print("Script complete")
else:
    print("Creating the observed and model dfs")
    # Set up the path to the ERA5 data
    # if the variable is tas
    if variable == "tas":
        # already regridded!
        obs_path = (
            "/gws/nopw/j04/canari/users/benhutch/ERA5/t2m_ERA5_regrid_HadGEM.nc"
        )
    # if the variable is sfcWind
    elif variable == "sfcWind":
        # needs regridding
        obs_path = "/gws/nopw/j04/canari/users/benhutch/ERA5/surface_wind_ERA5.nc"
    else:
        raise ValueError("Variable not recognised")

    # Load the model ensemble
    model_ds = funcs.load_model_data_xarray(
        model_variable=variable,
        model=model,
        experiment=experiment,
        start_year=first_year,
        end_year=last_year,
        first_fcst_year=int(first_year) + 1,
        last_fcst_year=int(first_year) + 2,
        months=months,
        frequency=freq,
        parallel=False,
    )

    # print that we have loaded the model data
    print("Loaded the model data")

    # # Get the size of the model data in bytes
    # size_in_bytes = model_ds[variable].size * model_ds[variable].dtype.itemsize

    # # Convert bytes to gigabytes
    # size_in_gb = size_in_bytes / (1024 ** 3)

    # # Print the size
    # print(f"Model data size: {size_in_gb} GB")

    # Modify member coordiante before conbersion to iris
    model_ds["member"] = model_ds["member"].str[1:-6].astype(int)

    # convert to an iris cube
    model_cube = model_ds[variable].squeeze().to_iris()

    # Load the observed data
    obs_ds = xr.open_mfdataset(
        obs_path,
        combine="by_coords",
        parallel=False,
        engine="netcdf4",
    )

    # Restrict the time to the region we are interested in
    obs_ds = obs_ds.sel(
        time=slice(
            f"{int(first_year)}-{months[0]}-01",
            f"{int(last_year) + 1}-{months[-1]}-31",
        )
    )

    # If expver is present in the observations
    if "expver" in obs_ds.coords:
        # Combine the first two expver variables
        obs_ds = obs_ds.sel(expver=1).combine_first(obs_ds.sel(expver=5))

    # # Get the size of the observed data in bytes
    # size_in_bytes = obs_ds[obs_var].size * obs_ds[obs_var].dtype.itemsize

    # # Convert bytes to gigabytes
    # size_in_gb = size_in_bytes / (1024 ** 3)

    # # Print the size
    # print(f"Observed data size: {size_in_gb} GB")

    # convert to an iris cube
    obs_cube = obs_ds[obs_var].squeeze().to_iris()

    # if the lats and lons are not the same
    if (
        not model_cube.coord("latitude").shape == obs_cube.coord("latitude").shape
        or not model_cube.coord("longitude").shape
        == obs_cube.coord("longitude").shape
    ):
        print("Regridding model data")
        # regrid the obs cube to the model cube
        obs_cube = obs_cube.regrid(model_cube, iris.analysis.Linear())

    # make sure the cubes are correct in -180 to 180 lons
    obs_cube = obs_cube.intersection(longitude=(-180, 180))
    model_cube = model_cube.intersection(longitude=(-180, 180))

    # create the mask
    MASK_MATRIX = funcs.create_masked_matrix(
        country=country,
        cube=model_cube,
    )

    # print the shape of the mask matrix
    print(f"Mask matrix shape: {MASK_MATRIX.shape}")

    # print the sum of the mask matrix
    print(f"Mask matrix sum: {np.sum(MASK_MATRIX)}")

    # Apply the mask to the observed data
    obs_values = obs_cube.data * MASK_MATRIX
    model_values = model_cube.data * MASK_MATRIX

    # Where there are zeros in the mask we want to set these to Nans
    obs_values_masked = np.where(MASK_MATRIX == 0, np.nan, obs_values)
    model_values_masked = np.where(MASK_MATRIX == 0, np.nan, model_values)

    # Take the Nanmean of the data
    obs_values = np.nanmean(obs_values_masked, axis=(1, 2))
    model_values = np.nanmean(model_values_masked, axis=(3, 4))

    # Set up the ref time for the observations
    ref_time_obs = datetime(1900, 1, 1)

    # Extract the obs time points
    obs_time_points = obs_cube.coord("time").points

    # convert to obs datetimes
    obs_datetimes = [
        ref_time_obs + timedelta(hours=int(tp)) for tp in obs_time_points
    ]

    # Set up a dataframe for the observations
    obs_df = pd.DataFrame(
        {
            "time": obs_datetimes,
            "obs": obs_values,
        }
    )

    # set up an empty df for the model data
    model_df = pd.DataFrame()

    # extract the init, member and lead time points
    init_years = model_cube.coord("init").points
    members = model_cube.coord("member").points
    lead_times = model_cube.coord("lead").points

    # loop through the inits, members and leadtimes
    for i, init_year in enumerate(init_years):
        for m, member in enumerate(members):
            for l, lead_time in enumerate(lead_times):
                # get the model data
                model_data = model_values[i, m, l]

                # set up the model df this
                model_df_this = pd.DataFrame(
                    {
                        "init_year": [init_year],
                        "member": [member],
                        "lead": [lead_time],
                        "data": [model_data],
                    },
                )

                # concat to the model df
                model_df = pd.concat([model_df, model_df_this])

    # print the head of the obs df
    print(obs_df.head())

    # print the head of the model df
    print(model_df.head())

    # save the dfs
    if not os.path.exists(output_dir_dfs):
        os.makedirs(output_dir_dfs)

    # save the obs df
    if not os.path.exists(obs_df_path):
        print("Saving the observed df")
        obs_df.to_csv(obs_df_path, index=False)

    # save the model df
    if not os.path.exists(model_df_path):
        print("Saving the model df")
        model_df.to_csv(model_df_path, index=False)

: 

In [ ]:
# constrain the obs df to only months 10, 11, 12, 1, 2, 3
# esnure that the time is a datetime
obs_df["time"] = pd.to_datetime(obs_df["time"])

# set the time as the index for the obs df
obs_df.set_index("time", inplace=True)

# # remove the name of the index
# obs_df.index.name = None

# print the head of the obs df
print(obs_df.head())

# constrain to the months
obs_df = obs_df[obs_df.index.month.isin(months)]

# NOTE: Not taking ONDJFM averages
# if months contains 12, 1 in sequence
# if 12 in months and 1 in months:
#     # shift back by months and take the annual mean
#     obs_df = obs_df.shift(-int(months[-1])).resample("A").mean()

# if there are any Nans in the obs df, drop them
obs_df.dropna(inplace=True)

# set up time as a column
obs_df.reset_index(inplace=True)

: 

In [ ]:
# create a new model df for subsetting to first ONDJFM
model_df_ondjfm = pd.DataFrame()

# turn leads into a list of ints
if lead_year != "9999":
    if "-" in lead_year:
        leads = list(
            range(
                int(lead_year.split("-")[0]),
                int(lead_year.split("-")[1]) + 1,
            )
        )
    else:
        leads = [int(lead_year)]

    # print the leads to extract
    print(f"Leads to extract: {leads}")
elif lead_year == "9999":
    # Set up the leads to extract list range 1-10
    leads = list(range(1, 11))
else:
    raise ValueError("Lead year not recognised")

: 

In [ ]:
# loop over the unique init years and members in model_df
for init_year in model_df["init_year"].unique():
    for member in model_df["member"].unique():
        for l in leads:
            # extract the model data
            model_data = model_df[
                (model_df["init_year"] == init_year)
                & (model_df["member"] == member)
            ]

            # create the list of lead months to extract
            lead_months_year_base = [l * lead_months[0] for lm in lead_months]

            # # print the lead months year base
            # print("lead months year base:", lead_months_year_base)

            # create the list of lead months to extract
            for i in range(len(lead_months_year_base)):
                lead_months_year_base[i] = lead_months_year_base[i] + i

            # # print the lead months year base
            # print("lead months year base:", lead_months_year_base)

            # # subset to lead values [12, 13, 14, 15, 16, 17] and take the mean
            # # first complete ONDJFM season
            # # FIXME: Hardcoded for now
            # model_data = model_data[model_data["lead"].isin(lead_months_year_base)]

            # mean_data = model_data["data"].mean()
                
            # # print lead months year base
            # print("lead months year base:", lead_months_year_base)

            # loop over the lead months
            for lm in lead_months_year_base:
                # subset to the lead month
                mean_data = model_data[model_data["lead"] == lm].mean()["data"]

                # create a dataframe this
                model_data_this = pd.DataFrame(
                    {
                        "init_year": [init_year],
                        "member": [member],
                        "lead": [lm],
                        "data": [mean_data],
                    }
                )

                model_df_ondjfm = pd.concat([model_df_ondjfm, model_data_this])

: 

In [ ]:
# if the detrend is True
if detrend and bias_correct == "None":
    print("Detrending the data, no bias correction")

    # apply the function to detrend the data
    obs_df, model_df_ondjfm = funcs.apply_detrend(
        obs_df=obs_df,
        model_df=model_df_ondjfm,
        obs_val_name="obs",
        model_val_name="data",
        obs_time_name="time",
        model_time_name="init_year",
        model_member_name="member",
        model_lead_name="lead",
    )

    # Set up the name for the obs val name
    obs_val_name = "obs_dt"
    model_val_name = "data_dt"
elif bias_correct != "None" and not detrend:
    print("Bias correcting the data, no detrending")

    # if the bias correction is linear_scaling
    if bias_correct == "linear_scaling":
        # apply the function to bias correct the data
        model_df_ondjfm = funcs.bc_linear_scaling(
            obs_df=obs_df,
            model_df=model_df_ondjfm,
            obs_val_name="obs",
            model_val_name="data",
        )
    elif bias_correct == "variance_scaling":
        # apply the function to bias correct the data
        model_df_ondjfm = funcs.bc_variance_scaling(
            obs_df=obs_df,
            model_df=model_df_ondjfm,
            obs_val_name="obs",
            model_val_name="data",
        )
    elif bias_correct == "quantile_mapping":
        # Use James functions to correct the model data
        qm_adjustment = ba.QMBiasAdjust(
            obs_data = obs_df["obs"],
            mod_data = model_df_ondjfm["data"],
        )

        # assign the corrected data to the model df
        model_df_ondjfm["data_bc"] = qm_adjustment.correct()
    elif bias_correct == "quantile_delta_mapping":
        # Use James functions to correct the model data
        qdm_adjustment = ba.QDMBiasAdjust(
            obs_data = obs_df["obs"],
            mod_data = model_df_ondjfm["data"],
        )

        # assign the corrected data to the model df
        model_df_ondjfm["data_bc_qdm"] = qdm_adjustment.correct()

        # compare to the quantile mapping adjustment
        qm_adjustment = ba.QMBiasAdjust(
            obs_data = obs_df["obs"],
            mod_data = model_df_ondjfm["data"],
        )

        # assign the corrected data to the model df
        model_df_ondjfm["data_bc_qm"] = qm_adjustment.correct()

        # take the difference between the two columns
        model_df_ondjfm["data_bc_diff"] = model_df_ondjfm["data_bc_qm"] - model_df_ondjfm["data_bc_qdm"]

        # print the head of the model df
        print(model_df_ondjfm.head())

        # print the tail of the model df
        print(model_df_ondjfm.tail())
    elif bias_correct == "scaled_distribution_mapping":
        print("Applying scaled distribution mapping")

        sdm_adjustment = ba.SDMBiasAdjust(
            obs_data = obs_df["obs"],
            mod_data = model_df_ondjfm["data"],
        )

        # assign the corrected data to the model df
        model_df_ondjfm["data_bc"] = sdm_adjustment.correct()
    else:
        print(f"Bias correction method {bias_correct} not recognised")

    # Set up the name for the obs val name
    obs_val_name = "obs"
    model_val_name = "data_bc"

    # print the mean bias
    print(
        "Mean bias:",
        np.mean(model_df_ondjfm[model_val_name]) - np.mean(obs_df[obs_val_name]),
    )

    # print the spread bias
    print(
        "Spread bias:",
        np.std(model_df_ondjfm[model_val_name]) - np.std(obs_df[obs_val_name]),
    )

elif bias_correct != "None" and detrend:
    print("Bias correcting the data and detrending")

    # apply the function to detrend the data
    obs_df, model_df_ondjfm = funcs.apply_detrend(
        obs_df=obs_df,
        model_df=model_df_ondjfm,
        obs_val_name="obs",
        model_val_name="data",
        obs_time_name="time",
        model_time_name="init_year",
        model_member_name="member",
        model_lead_name="lead",
    )

    # # print the mean of the model data
    # print("Model data mean before bias correction:", np.mean(model_df_ondjfm["data_dt"]))

    # # print the spread of the model data
    # print("Model data spread before bias correction:", np.std(model_df_ondjfm["data_dt"]))

    if bias_correct == "linear_scaling":
        # apply the function to bias correct the data
        model_df_ondjfm = funcs.bc_linear_scaling(
            obs_df=obs_df,
            model_df=model_df_ondjfm,
            obs_val_name="obs_dt",
            model_val_name="data_dt",
        )
    elif bias_correct == "variance_scaling":
        # apply the function to bias correct the data
        model_df_ondjfm = funcs.bc_variance_scaling(
            obs_df=obs_df,
            model_df=model_df_ondjfm,
            obs_val_name="obs_dt",
            model_val_name="data_dt",
        )
    elif bias_correct == "quantile_mapping":
        # use James' functions to correct the model data
        qm_adjustment = ba.QMBiasAdjust(
            obs_data = obs_df["obs"],
            mod_data = model_df_ondjfm["data_dt"],
        )

        # assign the corrected data to the model df
        model_df_ondjfm["data_dt_bc"] = qm_adjustment.correct()
    elif bias_correct == "quantile_delta_mapping":
        # Use James functions to correct the model data
        qdm_adjustment = ba.QDMBiasAdjust(
            obs_data = obs_df["obs"],
            mod_data = model_df_ondjfm["data_dt"],
        )

        # assign the corrected data to the model df
        model_df_ondjfm["data_dt_bc"] = qdm_adjustment.correct()
    elif bias_correct == "scaled_distribution_mapping":
        print("Applying scaled distribution mapping")

        sdm_adjustment = ba.SDMBiasAdjust(
            obs_data = obs_df["obs"],
            mod_data = model_df_ondjfm["data_dt"],
        )

        # assign the corrected data to the model df
        model_df_ondjfm["data_dt_bc"] = sdm_adjustment.correct()
    else:
        print(f"Bias correction method {bias_correct} not recognised")
        sys.exit()

    # # print the mean of the model data
    # print("Model data mean after bias correction:", np.mean(model_df_ondjfm["data_dt_bc"]))

    # # print the spread of the model data
    # print("Model data spread after bias correction:", np.std(model_df_ondjfm["data_dt_bc"]))

    # # print the observed mean
    # print("Observed data mean before bias correction:", np.mean(obs_df["obs_dt"]))

    # # print the spread of the observed data
    # print("Observed data spread before bias correction:", np.std(obs_df["obs_dt"]))

    # sys.exit()

    # Set up the name for the obs val name
    obs_val_name = "obs_dt"
    model_val_name = "data_dt_bc"

else:
    obs_val_name = "obs"
    model_val_name = "data"

: 

In [ ]:
# print the obs val name being used
print("----------------")
print(f"Obs val name: {obs_val_name}")
print(f"Model val name: {model_val_name}")
print("----------------")

: 